<span style="font-family:New York Times; font-size:1.5em; color:green;"> 
pika usage

* [https://www.rabbitmq.com/tutorials/tutorial-one-python.html](https://www.rabbitmq.com/tutorials/tutorial-one-python.html)
* [https://www.cloudamqp.com/blog/2015-05-21-part2-3-rabbitmq-for-beginners_example-and-sample-code-python.html](https://www.cloudamqp.com/blog/2015-05-21-part2-3-rabbitmq-for-beginners_example-and-sample-code-python.html)

In [ ]:
!pip install pika 

* server: `rabbitmq-server`
* client: pika
Without a server, the next step is doomed!

In [ ]:
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
# We're connected now, to a broker on the local machine 
channel = connection.channel()
# create a channel in the TCP connection.

# Publisher

## `channel.queue_declare`


In [ ]:
channel.queue_declare(queue = 'hello') 
# create a hello queue, a recipient queue, to which the message will be delivered:

## `channel.basic_publish`

In [ ]:
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
# use a default exchange identified by an empty string
# This exchange is special ‒ it allows us to specify exactly to which queue the message should go.
print(" [x] Sent 'Hello World!'")
chanel.closed()
# The connection will be closed after the message has been published.

<span style = "font-family: New York Times; font-size:1.2em; color:green;">
The default exchange means that messages are routed to the queue with the name specified by routing_key, if it exists. (The default exchange is a direct exchange with no name).

# Receiver

## callback(ch, method, properties, body)

<span style="font-family:New York Times; font-size:1.2em; color:green;"> 
Receiving messages from the queue is more complex. It works by subscribing a callback function to a queue. Whenever we receive a message, this callback function is called by the Pika library.

In [ ]:
# #create a function which is called on incoming messages
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

<span style="font-family:New York Times; font-size:1.2em; color:green;"> 
Next, we need to tell RabbitMQ that this particular callback function should receive messages from our hello queue:

In [ ]:
channel.basic_consume(queue='hello',
                      auto_ack=True,
                      on_message_callback=callback)

## `channel.start_consuming()`
<span style="font-family:New York Times; font-size:1.2em; color:green;"> 
And finally, we enter a never-ending loop that waits for data and runs callbacks whenever necessary.

## Temporary test

In [ ]:
class MessageQueue(object):
    def __init__(self, code, message, data):
        self.code = code
        self.message = message
        self.data = data

    def get_json(self):
        return json.dumps({'code': self.code, 'msg': self.message, 'data': self.data})